In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd

data = pd.read_csv("/content/gdrive/My Drive/Аудиоаналитика. Лабы/Train set/train.csv", delimiter=',')

In [0]:
import pickle

def load_pkl(name):
    with open('/content/gdrive/My Drive/Аудиоаналитика. Лабы/Train set/' + name + '.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

In [0]:
X = load_pkl('X_sr8000_hl4000_nfft8000') # 'X_sr8000_hl8000_nfft8000'
y = load_pkl('y')

In [0]:
uniq_labels = np.sort(np.unique(y))
label_to_id = {label: i for i, label in enumerate(uniq_labels)}
id_to_label = [label for label, i in label_to_id.items()]

y_re = [label_to_id[label] for label in y]

In [0]:
import numpy as np

new_size = 61 #или 30 с чем то (32?)

X_re = []
for x in X:
    a = np.zeros((128, new_size))
    a[:x.shape[0], :x.shape[1]] = x
    X_re.append(a)

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_re, y_re, test_size=0.30, random_state=42)

In [0]:
from torch.utils.data import Dataset


class AudioDataset(Dataset):
    def __init__(self, x, y=None):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.x[idx], self.y[idx]
        return self.x[idx]

In [0]:
trainset = AudioDataset(x_train, y_train)
testset = AudioDataset(x_test, y_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = id_to_label

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 29 * 12, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 41)

    def forward(self, x):
        x = x.view(-1, 1, 128, 61) 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 16 * 29 * 12)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [0]:
random_input = torch.randn(1, 1, 128, 61, requires_grad=True)
net = Net()
output = net(random_input)
print(output)

tensor([[-0.0347, -0.0643, -0.0011,  0.0628,  0.0782, -0.1629, -0.0192,  0.0931,
          0.1044,  0.1019]], grad_fn=<AddmmBackward>)


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
running_loss = 0.0
for i, data in enumerate(trainloader, 0):
    # get the inputs
    inputs, labels = data

    print(inputs.shape, labels)

In [0]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 3.543
[1,   400] loss: 3.557
[1,   600] loss: 3.586
[1,   800] loss: 3.532
[2,   200] loss: 3.512
[2,   400] loss: 3.482
[2,   600] loss: 3.503
[2,   800] loss: 3.521
Finished Training


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 9 %


In [0]:
count = 0

class_correct = list(0. for i in range(41))
class_total = list(0. for i in range(41))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.float())
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        count += 4
        if count == 1708: # вынужденная мера
            label = labels[0]
            class_correct[label] += int(c)
            class_total[label] += 1
        else:     
            for i in range(4):
                label = labels[i]
                class_correct[label] += int(c[i])
                class_total[label] += 1


for i in range(41):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Acoustic_guitar : 56 %
Accuracy of Applause : 93 %
Accuracy of  Bark :  0 %
Accuracy of Bass_drum : 79 %
Accuracy of Burping_or_eructation :  0 %
Accuracy of   Bus :  0 %
Accuracy of Cello : 25 %
Accuracy of Chime :  0 %
Accuracy of Clarinet :  9 %
Accuracy of Computer_keyboard :  0 %
Accuracy of Cough :  0 %
Accuracy of Cowbell :  0 %
Accuracy of Double_bass : 12 %
Accuracy of Drawer_open_or_close :  0 %
Accuracy of Electric_piano :  0 %
Accuracy of  Fart :  0 %
Accuracy of Finger_snapping :  0 %
Accuracy of Fireworks :  5 %
Accuracy of Flute : 34 %
Accuracy of Glockenspiel :  0 %
Accuracy of  Gong :  0 %
Accuracy of Gunshot_or_gunfire :  0 %
Accuracy of Harmonica :  0 %
Accuracy of Hi-hat :  0 %
Accuracy of Keys_jangling :  0 %
Accuracy of Knock :  0 %
Accuracy of Laughter :  0 %
Accuracy of  Meow :  0 %
Accuracy of Microwave_oven :  0 %
Accuracy of  Oboe :  0 %
Accuracy of Saxophone :  0 %
Accuracy of Scissors :  0 %
Accuracy of Shatter :  0 %
Accuracy of Snare_drum :  0